In [1]:
import clifford_opt as opt

In [2]:
import os
import sys
from qiskit import QuantumCircuit, transpile
from qiskit.providers.aer import AerSimulator
import numpy as np
import pandas as pd
from qiskit.quantum_info import Operator
from tqdm import tqdm

In [3]:
directory_path='QASMBench/small/'

In [4]:
def remove_conditions(circuit : QuantumCircuit):
    circ = QuantumCircuit(*circuit.cregs, *circuit.qregs)
    circ.name = circuit.name
    for instr, qargs, cargs in circuit.data:
        if (instr.condition is None and instr.name != 'measure' and instr.name != 'reset' and instr.name != 'barrier'):
            circ.append(instr, qargs, cargs)
    return circ

In [5]:
paths = []
for root, dirs, files in os.walk(directory_path):
    for dir_name in dirs:
        qasm_file_path = os.path.join(directory_path, dir_name, f"{dir_name}.qasm")
        number_at_end = int(''.join(filter(str.isdigit, dir_name)))
        if os.path.exists(qasm_file_path) and number_at_end <= 20:
            paths.append(qasm_file_path)
paths

['QASMBench/small/qft_n4/qft_n4.qasm',
 'QASMBench/small/fredkin_n3/fredkin_n3.qasm',
 'QASMBench/small/dnn_n2/dnn_n2.qasm',
 'QASMBench/small/qec_sm_n5/qec_sm_n5.qasm',
 'QASMBench/small/deutsch_n2/deutsch_n2.qasm',
 'QASMBench/small/bell_n4/bell_n4.qasm',
 'QASMBench/small/teleportation_n3/teleportation_n3.qasm',
 'QASMBench/small/shor_n5/shor_n5.qasm',
 'QASMBench/small/qpe_n9/qpe_n9.qasm',
 'QASMBench/small/wstate_n3/wstate_n3.qasm',
 'QASMBench/small/vqe_uccsd_n4/vqe_uccsd_n4.qasm',
 'QASMBench/small/lpn_n5/lpn_n5.qasm',
 'QASMBench/small/hhl_n10/hhl_n10.qasm',
 'QASMBench/small/sat_n7/sat_n7.qasm',
 'QASMBench/small/hhl_n7/hhl_n7.qasm',
 'QASMBench/small/qrng_n4/qrng_n4.qasm',
 'QASMBench/small/qaoa_n6/qaoa_n6.qasm',
 'QASMBench/small/pea_n5/pea_n5.qasm',
 'QASMBench/small/toffoli_n3/toffoli_n3.qasm',
 'QASMBench/small/dnn_n8/dnn_n8.qasm',
 'QASMBench/small/adder_n10/adder_n10.qasm',
 'QASMBench/small/inverseqft_n4/inverseqft_n4.qasm',
 'QASMBench/small/variational_n4/variational

In [6]:
circuits = []
for path in tqdm(paths):    
    filename = path.split('/')[-1]
    sys.stdout.flush()
    circuit = QuantumCircuit.from_qasm_file(path)
    if len(circuit) < 1000:
        circuit.name = filename[:-5]
        circuit = remove_conditions(circuit)
        #print(circuit.name)
        #circuit.remove_final_measurements()
        circuit2 = transpile(circuit, optimization_level=0, basis_gates=opt.native_gates)
        circuit2.name = circuit.name
        circuits.append(circuit2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 39/39 [00:01<00:00, 33.34it/s]


In [7]:
def sorting_key(qc):
    return (qc.num_qubits, len(qc))

circuits_sorted = sorted(circuits, key=sorting_key)

In [8]:
# data = []
# shots = 10
# for circuit in circuits_sorted:
#     print('\\')
#     print(circuit.name)
#     shots = 10
#     eps = 1
#     fid1 = fidelity_density_matrix_orig(circuit, measure=True)
#     while shots < 2000:
#         epses = []
#         k = 0
#         while (k < 2):
#             fid2 = fidelity_clifford_measuring(circuit, shots=shots, use_tqdm=False)
#             data.append({'circuit' : circuit.name, 
#                    'orig_fidelity' : fid1, 
#                    'clifford_fidelity' : fid2, 
#                     'shots': shots})
#             eps = np.abs(fid2 - fid1) / fid1
#             epses.append(eps)
#             #print(shots, circuit.name, fid1, fid2, eps)
#             print(shots, end= ',')
#             k += 1
#         eps = np.max(epses) if len(epses) else eps
#         shots = int(shots * 1.3)

In [9]:
# df = pd.DataFrame(data)
# df['qubit_count'] = df['circuit'].apply(lambda c: int(c[-1]))
# df['eps'] = np.abs(df['orig_fidelity'] - df['clifford_fidelity']) / df['orig_fidelity']
# lens = {c.name : len(c) for c in circuits_sorted}
# circs = {c.name : c for c in circuits_sorted}
# def peaks(c1 : QuantumCircuit):
#     c = c1.copy()
#     g = AerSimulator()
#     c.measure_all()
#     return len(g.run(c, shots=2000).result().data()['counts'])
# peakss = {c.name: peaks(c) for c in circuits_sorted}
# df['len'] = df['circuit'].apply(lambda c: lens[c])
# df['peaks'] = df['circuit'].apply(lambda c: peakss[c])

In [10]:
def AssertCircuit(c, res_c):
    u = Operator(res_c).to_matrix()
    v = Operator(c).to_matrix()
    assert(np.abs(np.abs((u @ v.conj().T).trace() / u.shape[0]) - 1) < 1e-6)

In [11]:
from time import time
import sys
from importlib import reload
reload(opt)

<module 'clifford_opt' from '/Users/nolongerlaugh/mur/edc/CliffordOpt/clifford_opt/__init__.py'>

In [ ]:
fs_merge = []
res_opt = []
fs_opt = []
for c1 in circuits_sorted:
    print(f'-----{c1.name}-------')
    print("merge_unitary...")
    c = opt.transformer.merge_unitary(c1, u=True)
    fs_merge.append(opt.fidelityQ.Qfidelity_density_matrix_orig(c))
    print("transpile...")
    res_c = opt.transpiler(qc=c, func=opt.fidelityQ.Qfidelity_density_matrix_clifford, 
                           n_iter=200000, same=100000, use_tqdm=True, output=False)
    res_opt.append(res_c)
    fs_opt.append(opt.fidelityQ.Qfidelity_density_matrix_orig(res_c))
    print(fs_merge[-1], fs_opt[-1])

-----deutsch_n2-------
merge_unitary...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 1567.83it/s]


transpile...


 21%|█████████████████████                                                                             | 42887/200000 [06:19<22:35, 115.87it/s]